### Multinomial logit models

Here, we are using models that incorporate the comparison of attributes across alternatives and can also be used to capture heterogenity in the choice modelling process.

In [1]:
safety <- read.csv("csv/safetydata.csv")
str(safety)
# note that this dataframe has its columns as observations instead of rows


table(safety$Choice)

'data.frame':	9500 obs. of  112 variables:
 $ Case      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ No        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Task      : int  1 2 3 4 5 6 7 8 9 10 ...
 $ CC1       : int  0 3 1 0 0 3 0 3 3 0 ...
 $ GN1       : int  1 0 1 0 2 0 2 0 0 0 ...
 $ NS1       : int  4 0 5 0 2 0 0 1 0 0 ...
 $ BU1       : int  6 0 0 2 0 5 0 0 3 0 ...
 $ FA1       : int  0 2 2 0 1 0 1 0 0 0 ...
 $ LD1       : int  0 2 0 3 2 0 0 2 0 0 ...
 $ BZ1       : int  0 2 3 0 0 1 2 0 2 0 ...
 $ FC1       : int  0 1 0 2 2 0 0 0 2 2 ...
 $ FP1       : int  3 0 1 0 0 4 0 2 0 1 ...
 $ RP1       : int  1 0 0 1 0 0 1 1 0 2 ...
 $ PP1       : int  1 0 0 0 2 2 1 0 2 0 ...
 $ KA1       : int  0 1 0 0 1 2 0 1 2 0 ...
 $ SC1       : int  0 3 1 0 0 4 4 0 0 1 ...
 $ TS1       : int  1 0 2 0 0 1 0 0 1 1 ...
 $ NV1       : int  1 0 0 3 3 0 0 0 3 3 ...
 $ MA1       : int  4 0 0 4 0 0 1 1 2 0 ...
 $ LB1       : int  0 4 0 4 3 0 0 2 0 4 ...
 $ AF1       : int  0 2 0 2 0 1 3 0 0 2 ...
 $ HU1       : int  0 0 2 1 0 0


 Ch1  Ch2  Ch3  Ch4 
2165 2404 2136 2795 

In [9]:
library(mlogit)

S <- mlogit.data(subset(safety, Task<=12),
                 shape = "wide",
                 choice = "Choice",
                 sep = "",
                 varying = c(4:83),
                 alt.levels = c("Ch1", "Ch2", "Ch3", "Ch4"),
                 id.var = "Case")

# no closed-form solution for the mlogit model, just like logistic regression
M <- mlogit(Choice~CC+GN+NS+BU+FA+LD+BZ+FC+FP+RP+PP+KA+SC+TS+NV+MA+LB+AF+HU+Price-1, data=S)
summary(M)


Call:
mlogit(formula = Choice ~ CC + GN + NS + BU + FA + LD + BZ + 
    FC + FP + RP + PP + KA + SC + TS + NV + MA + LB + AF + HU + 
    Price - 1, data = S, method = "nr", print.level = 0)

Frequencies of alternatives:
    Ch1     Ch2     Ch3     Ch4 
0.22850 0.25500 0.23567 0.28083 

nr method
4 iterations, 0h:0m:0s 
g'(-H)^-1g = 2.05E-07 
gradient close to zero 

Coefficients :
       Estimate Std. Error  t-value  Pr(>|t|)    
CC     0.108533   0.020754   5.2295 1.699e-07 ***
GN     0.067709   0.029925   2.2626 0.0236592 *  
NS     0.021263   0.013039   1.6307 0.1029606    
BU     0.026681   0.010931   2.4408 0.0146541 *  
FA     0.052062   0.029611   1.7582 0.0787097 .  
LD     0.057242   0.020777   2.7550 0.0058689 ** 
BZ     0.040601   0.020933   1.9395 0.0524345 .  
FC     0.056979   0.029856   1.9084 0.0563338 .  
FP     0.008004   0.015974   0.5011 0.6163261    
RP     0.058425   0.029830   1.9586 0.0501622 .  
PP     0.038776   0.021007   1.8459 0.0649103 .  
KA     0.122004

In [4]:
# obtaining log-likelihood for a perfectly random baseline model
# x6000 because there are 6000 observations total
# 1/4 because there are 4 alternatives
6000*log(1/4)

[1] -8317.766

Now, predicting the choices that are made.

In [5]:
# get actual choices
ActualChoice <- subset(safety, Task<=12)[,"Choice"]
str(ActualChoice)

# get predicted choices
F <- predict(M, newdata=S)
PredictedChoice <- apply(F,1,which.max)

# accuracy matrix
table(PredictedChoice,ActualChoice)
#accuracy
(616+647+629+700)/6000

 Factor w/ 4 levels "Ch1","Ch2","Ch3",..: 2 1 2 2 2 2 2 2 2 2 ...


               ActualChoice
PredictedChoice Ch1 Ch2 Ch3 Ch4
              1 616 273 226 356
              2 222 647 235 320
              3 194 258 629 309
              4 339 352 324 700

[1] 0.432

Now, predicting choices on testset.

In [6]:
Test <- mlogit.data(subset(safety, Task>12),
                 shape = "wide",
                 choice = "Choice",
                 sep = "",
                 varying = c(4:83),
                 alt.levels = c("Ch1", "Ch2", "Ch3", "Ch4"),
                 id.var = "Case")

# predicting on test set
TestPredict <- predict(M, Test)
TestPredictedChoice <- apply(TestPredict,1,which.max)
TestActualChoice <- subset(safety, Task>12)[,"Choice"]
table(TestPredictedChoice, TestActualChoice)

#accuracy
(376+435+356+517)/3500

                   TestActualChoice
TestPredictedChoice Ch1 Ch2 Ch3 Ch4
                  1 376 120  93 176
                  2 119 435  97 227
                  3 118 124 356 190
                  4 181 195 176 517

[1] 0.4811429

How much is the price willingness that the consumer is wiling to pay for a unit increase in one of the features, all else held constant?

Suppose the following two:
- $\large U_1 = \beta_1x_1 + \beta_{price}x_{p} + ...$
- $\large U_2 = \beta_1(x_1+\delta_1) + \beta_{price}({x_{price}+\delta_p}) + ...$

If we wish to find the willingness to pay (WTP), set $U_1=U_2$. Rationale for doing so is to find out the "exchange rate" bewteen an additional unit of feature 1 and the money the consumer is willing to pay, hence WTP. We can find out how much $\delta_p$ will change if we change feature $x_1$ by $\delta_1$.

### Mixed logit models

However, we usually assume that the $\beta$ values are fixed in the multinomial model. What if each consumer(observation) has a different subjective $\beta$ values? Then, we will need to consider the distribution of those consumers(observations) then.

**Standard logit model**
$$\Large P(Y_i=k)= \frac{e^{\beta'x_{ik}}}{\sum_{l=1}^{k}e^{\beta'x_{il}}}$$

**Mixed logit model**
$$\Large P(Y_i=k)= \int{\frac{e^{\beta'x_{ik}}}{\sum_{l=1}^{k}e^{\beta'x_{il}}}f(\beta)\:\:\:d\beta}$$
where $\widetilde{\beta} \sim f(\beta|\theta)$ for $\tilde{U_{ik}}= \tilde{\beta'}x_{ik}+\tilde{\epsilon_{ik}}$

### Panel data

If there are multiple observations of a consumer, how do we build a better model that has the ability to consider multicollinearity between $\beta$ variables.

$$\Large P(Y_i=k_1\:in\:test\:1 \cap Y_i=k_2\:in\:test\:2)=
\int{\frac{e^{\beta'x_{ik_1}}}{\sum_{l=1}^{k_1}e^{\beta'x_{il}}}
{\frac{e^{\beta'x_{ik_2}}}{\sum_{l=1}^{k_2}e^{\beta'x_{il}}}}
f(\beta)\:\:\:d\beta}$$

In [3]:
# now estimating mixed logit (may take some time)
M1 <- mlogit(Choice~CC+GN+NS+BU+FA+LD+BZ+FC+FP+RP+PP+KA+SC+TS+NV+MA+LB+AF+HU+Price-1,
             rpar = c(CC='n', GN='n', NS='n', BU='n', FA='n', LD='n', BZ='n', FC='n',
                      FP='n', RP='n', PP='n', KA='n', SC='n', TS='n', NV='n', MA='n',
                      LB='n', AF='n', HU='n', Price='n'),
             data = S, panel = T, print.level = F)
summary(M1)

ERROR: Error in mlogit(Choice ~ CC + GN + NS + BU + FA + LD + BZ + FC + FP + : could not find function "mlogit"


In [8]:
# recall that "Task" is simply the index of the observation of the customer
# Each customer has 12 observations (12 cars for valuation)

# predict on training set
P1 <- predict(M1, S)
PredictedChoice1 <- apply(P1, 1, which.max)
table(PredictedChoice1, ActualChoice)
#accuracy on training set
(530+552+537+905)/6000

# predict on test set
TestPredict1 <- predict(M1, Test)
ActualChoice1 <- subset(safety, Task>12)[,'Choice']
PredictedChoice1 <- apply(TestPredict1, 1, which.max)
table(PredictedChoice1, ActualChoice1)
#accuracy on test set
(331+377+316+644)/3500


                ActualChoice
PredictedChoice1 Ch1 Ch2 Ch3 Ch4
               1 530 243 179 288
               2 203 552 207 256
               3 173 220 537 236
               4 465 515 491 905

[1] 0.4206667

                ActualChoice1
PredictedChoice1 Ch1 Ch2 Ch3 Ch4
               1 331 116  77 137
               2 102 377  83 180
               3  98 110 316 149
               4 263 271 246 644

[1] 0.4765714

In [14]:
safety[1:20,"Task"]
str(safety)

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19  1

'data.frame':	9500 obs. of  112 variables:
 $ Case      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ No        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Task      : int  1 2 3 4 5 6 7 8 9 10 ...
 $ CC1       : int  0 3 1 0 0 3 0 3 3 0 ...
 $ GN1       : int  1 0 1 0 2 0 2 0 0 0 ...
 $ NS1       : int  4 0 5 0 2 0 0 1 0 0 ...
 $ BU1       : int  6 0 0 2 0 5 0 0 3 0 ...
 $ FA1       : int  0 2 2 0 1 0 1 0 0 0 ...
 $ LD1       : int  0 2 0 3 2 0 0 2 0 0 ...
 $ BZ1       : int  0 2 3 0 0 1 2 0 2 0 ...
 $ FC1       : int  0 1 0 2 2 0 0 0 2 2 ...
 $ FP1       : int  3 0 1 0 0 4 0 2 0 1 ...
 $ RP1       : int  1 0 0 1 0 0 1 1 0 2 ...
 $ PP1       : int  1 0 0 0 2 2 1 0 2 0 ...
 $ KA1       : int  0 1 0 0 1 2 0 1 2 0 ...
 $ SC1       : int  0 3 1 0 0 4 4 0 0 1 ...
 $ TS1       : int  1 0 2 0 0 1 0 0 1 1 ...
 $ NV1       : int  1 0 0 3 3 0 0 0 3 3 ...
 $ MA1       : int  4 0 0 4 0 0 1 1 2 0 ...
 $ LB1       : int  0 4 0 4 3 0 0 2 0 4 ...
 $ AF1       : int  0 2 0 2 0 1 3 0 0 2 ...
 $ HU1       : int  0 0 2 1 0 0